In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
from numpy import linalg as LA
import time
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import  KernelRidge
from sklearn.linear_model import  HuberRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic, Matern, ExpSineSquared,DotProduct
from sklearn.svm import SVR

from yellowbrick.features import ParallelCoordinates

In [ ]:
#Reading Input

df1 = pd.read_csv('Input.csv')

df1.rename(columns = {'TLI':'InletLiquidTemperature', 'TAI':'InletAirTemperature'
                   , 'PI':'InletPressure', 'H':'PackedBedHeight'
                   , 'D':'PackedBedDiameter', 'ML':'LiquidMassFlow'
                   , 'MA':'AirMassFlow', 'EPS':'PackedBedPorosity'}, inplace = True)

X = df1.to_numpy() #Input

#Reading Output

df2 = pd.read_csv('Output.csv')

df2.rename(columns = {'TLO':'OutletLiquidTemperature', 'TAO':'OutletAirTemperature'
                   , 'PO':'OutletPressure'}, inplace = True)

Y = df2.to_numpy()
Y0 = Y[:,0]

Y.shape
n,m=X.shape


In [ ]:
from sklearn.multioutput import MultiOutputRegressor
# Best linear estimation
time_start = time.time()
reg = MultiOutputRegressor(LinearRegression()).fit(X, Y)
time_elapsed = (time.time()-time_start)
print(time_elapsed)
Y_Lin_Predict=reg.predict(X)
MSE_Lin=mean_squared_error(Y,Y_Lin_Predict)
MAE_Lin=mean_absolute_error(Y,Y_Lin_Predict)
R2_Lin=r2_score(Y,Y_Lin_Predict)
Score_Lin=reg.score(X, Y)
print(MSE_Lin)
# print(MAE_Lin)
print(R2_Lin)
print(Score_Lin)

In [ ]:
kernel1 = RBF(10, (1e-3, 1e2))
kernel2 = RBF(5 , (1e-3, 1e2))
kernel3 = RationalQuadratic(length_scale=1.0, alpha=0.1)
kernel4 = Matern(length_scale=1.0, length_scale_bounds=(1e-05, 100000.0), nu=4.5)
kernel5 = ExpSineSquared(length_scale=2.0, periodicity=3.0,
                         length_scale_bounds=(1e-05, 100000.0),periodicity_bounds=(1e-05, 100000.0))
kernel6 = DotProduct()

gpr1 = GaussianProcessRegressor(kernel=kernel1, n_restarts_optimizer=10000,alpha=.1)
gpr2 = GaussianProcessRegressor(kernel=kernel2, n_restarts_optimizer=10000,alpha=.1)
gpr3 = GaussianProcessRegressor(kernel=kernel3, n_restarts_optimizer=10000,alpha=.1)
gpr4 = GaussianProcessRegressor(kernel=kernel4, n_restarts_optimizer=10000,alpha=.1)
gpr5 = GaussianProcessRegressor(kernel=kernel5, n_restarts_optimizer=10000,alpha=.1)
gpr6 = GaussianProcessRegressor(kernel=kernel6, n_restarts_optimizer=10000,alpha=.1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)
Y0_train = Y_train[:,0]
Y1_train = Y_train[:,1]
Y0_test = Y_test[:,0]
Y1_test = Y_test[:,1]

In [ ]:
my_stand_scaler_obj = StandardScaler()
my_stand_scaler_obj.fit(X_train)

# rescale the training data
X_stand_scaler = my_stand_scaler_obj.transform(X)

# rescale the training data
X_train_stand_scaler = my_stand_scaler_obj.transform(X_train)

# also, rescale the training data
X_test_stand_scaler = my_stand_scaler_obj.transform(X_test)


print(my_stand_scaler_obj.mean_)

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(gpr1,
                                                        X_stand_scaler, Y0,
                                                        cv = 10,
                                                        scoring="neg_mean_squared_error", 
                                                        train_sizes = np.linspace(0.1, 0.9, 16))

# Means and standard deviations of training set scores
train_mean1 = np.mean(train_scores, axis=1)
# Means and standard deviations of test set scores
test_mean1 = np.mean(test_scores, axis=1)

In [ ]:
test_mean1

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(gpr1,
                                                        X_stand_scaler, Y,
                                                        cv = 10,
                                                        scoring="r2",
                                                        train_sizes = np.linspace(0.1, 0.9, 16))

# Means and standard deviations of training set scores
train_mean2 = np.mean(train_scores, axis=1)
# Means and standard deviations of test set scores
test_mean2 = np.mean(test_scores, axis=1)

In [ ]:
gpr1 = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel1, n_restarts_optimizer=10000,alpha=.1))
gpr1.fit(X_train_scaled, Y_train)
#gpr1.score(X_train_scaled, Y_train)
gpr1.score(X_test_scaled, Y_test)


In [ ]:
gpr2 = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel2, n_restarts_optimizer=10000,alpha=.1))
gpr2.fit(X_train_scaled, Y_train)
#gpr2.score(X_train_scaled, Y_train)
gpr2.score(X_test_scaled, Y_test)


In [ ]:
gpr3 = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel3, n_restarts_optimizer=10000,alpha=.1))
gpr3.fit(X_train_scaled, Y_train)
#gpr3.score(X_train_scaled, Y_train)
gpr3.score(X_test_scaled, Y_test)

In [ ]:
gpr4 = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel4, n_restarts_optimizer=10000,alpha=.1))
gpr4.fit(X_train_scaled, Y_train)
#gpr4.score(X_train_scaled, Y_train)
gpr4.score(X_test_scaled, Y_test)

In [ ]:
gpr5 = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel5, n_restarts_optimizer=10000,alpha=.1))
gpr5.fit(X_train_scaled, Y_train)
#gpr5.score(X_train_scaled, Y_train)
gpr5.score(X_test_scaled, Y_test)

In [ ]:
gpr6 = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel6, n_restarts_optimizer=10000,alpha=.1))
gpr6.fit(X_train_scaled, Y_train)
#gpr6.score(X_train_scaled, Y_train)
gpr6.score(X_test_scaled, Y_test)